In [1]:
import numpy as np
import os

In [2]:
train_file_path = os.path.join("Data", "ES", "train")
test_file_path = os.path.join("Data", "ES", "dev.in")

train_words = []
train_tags = ["START"]

with open(train_file_path, "r", encoding="utf-8") as train_file:
    for l in train_file:
        if l.strip():  # Check if the line is not empty after stripping whitespace
            lst = l.split()
            x = " ".join(lst[:-1])  # Join all elements except the last as the word
            y = lst[-1]
            train_words.append(x)
            train_tags.append(y)
        else:
            train_tags.append("STOP")
            train_tags.append("START")
del train_tags[-1]

# Now you have train_words and tags populated with data


In [3]:
def estimate_transition_parameters(train_tags):
    transition_counts = {}  # To count transitions between states
    state_counts = {}  # To count occurrences of each state

    # Count transitions and state occurrences
    for i in range(len(train_tags) - 1):
        current_state = train_tags[i]
        next_state = train_tags[i + 1]

        if current_state not in state_counts:
            state_counts[current_state] = 0
        state_counts[current_state] += 1

        if current_state not in transition_counts:
            transition_counts[current_state] = {}
        if next_state not in transition_counts[current_state]:
            transition_counts[current_state][next_state] = 0
        transition_counts[current_state][next_state] += 1
    
    
    # Calculate transition probabilities
    transition_probs = {}
    for current_state, next_states in transition_counts.items():
        total_count = state_counts[current_state]
        transition_probs[current_state] = {next_state: count / total_count for next_state, count in next_states.items()}

    return transition_probs

In [4]:
### ALTERNATE EMISSION PROBABILITIES GENERATION ###
def estimate_emission_parameters(train_words, train_tags, k=1):
    edited_tags=[]
    for i in train_tags:
        if i!="START" and i!="STOP":
            edited_tags.append(i)

    emission_counts = {}  # To count emissions from states to words
    state_counts = {}  # To count occurrences of each state

    # Count emissions and state occurrences
    for i in range(len(train_words)):
        word = train_words[i]
        tag = edited_tags[i]

        if tag not in state_counts:
            state_counts[tag] = 0
        state_counts[tag] += 1

        if tag not in emission_counts:
            emission_counts[tag] = {}
        if word not in emission_counts[tag]:
            emission_counts[tag][word] = 0
        emission_counts[tag][word] += 1
    
    # Calculate emission probabilities with #UNK# handling
    emission_probs = {}
    for tag, word_counts in emission_counts.items():
        total_count = state_counts[tag]
        emission_probs[tag] = {}
        for word, count in word_counts.items():
            emission_probs[tag][word] = count / total_count
        emission_probs[tag]['#UNK#'] = k / (total_count + k)

    return emission_probs

In [5]:
emission_word_tag=estimate_emission_parameters(train_words,train_tags)
transition_parameters=estimate_transition_parameters(train_tags)

#print(emission_word_tag["O"])
print(transition_parameters["STOP"])

{'START': 1.0}


In [6]:
# import math
# def viterbi_algorithm_2(sentence, un_edited_states, transition_params, emission_params, ):
#     states=[]
#     for i in un_edited_states:
#         if i!="START" and i!="STOP" and i not in un_edited_states:
#             states.append(i)
#     n = len(sentence)
#     num_states = len(states)
#     viterbi = [{} for _ in range(n)]
#     backpointers = [{} for _ in range(n)]

#     # Initialization at time step 0
#     for state in states:
#         emission_prob = emission_params[state].get(sentence[0], emission_params[state]["#UNK#"])
#         print(emission_params)
#         viterbi[0][state] = math.log(transition_params['START'].get(state, 1e-10)) + math.log(emission_prob)
#         backpointers[0][state] = 'START'

#     # Forward pass
#     for t in range(1, n):
#         for state in states:
#             max_prob = float('-inf')
#             prev_state = None
#             for prev_state in states:
#                 transition_prob = transition_params[prev_state].get(state, 1e-10)
#                 emission_prob = emission_params[state].get(sentence[0], emission_params[state]["#UNK#"])
#                 prob = viterbi[t - 1].get(prev_state,1e-10) + math.log(transition_prob) + math.log(emission_prob)
#                 if prob > max_prob:
#                     max_prob = prob
#                     backpointers[t][state] = prev_state
#             viterbi[t][state] = max_prob

#     # Termination step
#     max_prob = float('-inf')
#     final_state = None
#     for state in states:
#         # print(viterbi[n - 1][state])
#         transition_prob = transition_params[state].get('STOP', 1e-10)
#         prob = viterbi[n - 1][state] + math.log(transition_prob)
#         if prob > max_prob:
#             max_prob = prob
#             final_state = state
#     print(backpointers)
#     print(viterbi)
#     # Backtracking step
#     best_path = [final_state]
#     for t in range(n - 1, 0, -1):
#         best_path.insert(0, backpointers[t][best_path[0]])

#     return best_path






In [10]:
def viterbi(sequence, train_tags, transition_parameters, emission_parameters):
    tags=[]
    for i in train_tags:
        if i!="START" and i!="STOP" and i not in tags:
            tags.append(i)
    n = len(sequence)
    num_tags = len(tags)

    LTR = np.zeros((num_tags, n))
    RTL = np.zeros((num_tags, n), dtype=int)

    for i in range(num_tags):
        emission_prob = emission_parameters[tags[i]].get(sequence[0], 1e-10)
        LTR[i,0] = np.log(transition_parameters['START'].get(tags[i], 1e-10)) + np.log(emission_prob)


    for w in range(1, n):
        for i in range(num_tags):
            max_prob = float('-inf')
            max_backpointer = -1000000
            for j in range(num_tags):
                transition_prob = transition_parameters[tags[j]].get(tags[i], 1e-10)
                emission_prob = emission_parameters[tags[i]].get(sequence[w], 1e-10)
                prob = LTR[j,w-1] + np.log(transition_prob) + np.log(emission_prob)
                if prob > max_prob:
                    max_prob = prob
                    max_backpointer = j

            LTR[i, w] = max_prob
            RTL[i, w] = max_backpointer

    stop_max_prob = float('-inf')
    stop_max_backpointer = -1000000
    for i in range(num_tags):
        transition_prob = transition_parameters[tags[i]].get('STOP', 1e-10)
        prob = LTR[i,n-1] + np.log(transition_prob)
        if prob > stop_max_prob:
            stop_max_prob = prob
            stop_max_backpointer = i

    print(RTL)
    # Retrieve the best path using backpointers
    best_path = []
    current_backpointer = stop_max_backpointer
    for w in range(n - 1, -1, -1):
        best_path.insert(0, tags[current_backpointer])
        if w == 0:
            break
        current_backpointer = RTL[current_backpointer, w]

    return best_path

In [11]:
# import numpy as np

# def viterbi(sequence, train_tags,transition_parameters, emission_parameters):
#     tags=[]
#     for i in train_tags:
#         if i!="START" and i!="STOP" and i not in tags:
#             tags.append(i)

#     n = len(sequence)
#     num_tags = len(tags)

#     LTR=np.zeros((num_tags,n))
#     RTL=np.zeros((num_tags,n))


#     first_word=sequence[0]
#     for i in range(num_tags):
#         try:
#             LTR[i,0]=transition_parameters["START"][tags[i]] 
#         except KeyError:
#             LTR[i,0]=1e-10
#         try:
#             LTR[i,0]*=emission_parameters[tags[i]][first_word]
#         except KeyError:
#             LTR[i,0]*=emission_parameters[tags[i]]["#UNK#"]

#     for w in range(1, n):
#         for i in range(0,num_tags):
#             max_prob = -1
#             max_backpointer = -1
#             for j in range(0,num_tags):
#                 try:
#                     prob = transition_parameters[tags[j]][tags[i]]
#                 except KeyError:
#                     prob = 1e-10
#                 try:
#                     prob*=emission_parameters[tags[i]][first_word]
#                 except KeyError:
#                     prob*=emission_parameters[tags[i]]["#UNK#"]

#                 if prob > max_prob:
#                     max_prob = prob
#                     max_backpointer = j
            
#             LTR[i, w] = max_prob
#             RTL[i, w] = max_backpointer
    
#     stop_max_prob = -1
#     stop_max_backpointer = -1
#     for i in range(num_tags):
#         try:
#             prob=LTR[i,n-1]*transition_parameters[tags[i]]["STOP"]
#         except KeyError:
#             LTR[i,n-1]*1e-10

#         if prob > stop_max_prob:
#             stop_max_prob = prob
#             stop_max_backpointer = i
    
#     # Retrieve the best path using backpointers
#     best_path = []
#     for w in range(n-1,-1,-1):
#         best_path.insert(0,tags[stop_max_backpointer])
#         if w==0:
#             break
#         stop_max_backpointer=int(RTL[stop_max_backpointer,w])

#     return best_path


In [13]:
test_file=open(test_file_path,"r")

pred_output_path = os.path.join(os.getcwd(), "Data", "ES", "dev.p2.out")
pred_output=open(pred_output_path,"w")
sequence=[]

for l in test_file:
    if l!="\n":
        sequence.append(l[0:-1])
    else:
        predicted_tags=viterbi(sequence,train_tags,transition_parameters,emission_word_tag)
        for i in range(0,len(predicted_tags)):
            pred_output.write(sequence[i]+" "+predicted_tags[i]+"\n")
        pred_output.write(l)
        pred_output.flush()
        sequence=[]

[[0 2 6 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 6 0 0 0 3 4 0 0 4 0 3]
 [0 1 5 0 0 1 1 5 1 0 1 1 1]
 [0 2 6 6 0 2 2 6 0 0 0 0 2]]
[[0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 4 3 4 3 0]
 [0 1 5 5 1 5 5 1]
 [0 2 6 6 2 6 6 2]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 3 4 4 0 0 3 3 0 3 4 0 3 4 0 0 3 3 0 0 0 3 4 3 0 3 0 3 4 0 0]
 [0 1 1 5 5 1 5 1 1 1 1 5 5 1 5 5 5 1 1 0 0 0 1 5 1 5 1 5 1 5 5 5]
 [0 2 2 6 6 2 6 2 6 6 2 6 6 2 6 6 0 2 2 0 0 0 2 6 2 6 2 6 2 6 6 2]]
[[0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 3 4 3 4 3]
 [0 1 1 5 1 5 5]
 [0 2 2 6 2 6 2]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [0 0 